# Reading data

In [1]:
import matplotlib.pyplot as plt
from datetime import datetime
import numpy as np
import os

In [2]:
import psycopg2
import pandas as pd

conn_string = "host='localhost' dbname='weather_env' user='postgres' password='postgres'"
conn = psycopg2.connect(conn_string)
cur = conn.cursor()
cur.execute("""SELECT * FROM environment_data.pivoted;""")
records = cur.fetchall()
cur.close()
df = pd.DataFrame(records, columns=[x[0] for x in cur.description])

# Basic statistics

In [ ]:
ddf = df.describe([.01, .10, .25, .5, .75, .90, .99]).T.sort_values('count')
ddf

# Creating normal plots

In [ ]:

index = pd.DatetimeIndex([datetime(y, m, d) for y, m, d in df[['measure_year', 'measure_month', 'measure_day']].values])

for name in df.columns:
    plt.plot(index, df[name])
    plt.title(name)
    plt.show()

In [ ]:
for name in df.columns[3:]:
    plt.hist(df[name], 100, density=True, histtype='step')
    plt.title(name)
    plt.show()

In [ ]:
for name in df.columns[3:]:
    # not equidistant so incorrect
    plt.magnitude_spectrum(df[name].dropna(), 1/24*3600)
    plt.title(name)
    plt.show()

# Correlation

With np.corrcoef

In [ ]:
#.drop(['measure_year', 'measure_month', 'measure_day'], axis=1)
corr_df = df.corr()
corr_df.loc[corr_df.abs().sort_values('arnhem_waterlevel').index, 'arnhem_waterlevel'].iloc[::-1]

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
im = ax.imshow(corr_df.values)
im.set_clim(-1, 1)

ax.xaxis.set(ticks=list(range(len(corr_df.columns))), ticklabels=corr_df.columns)
plt.xticks(rotation=90)
ax.yaxis.set(ticks=list(range(len(corr_df.columns))), ticklabels=corr_df.columns)

cbar = ax.figure.colorbar(im, ax=ax, format='% .2f')
plt.show()

# Create correlation Shift

With np.correlate full, doesn't make any sense when the the timestep is unequidistant

In [ ]:
shift_df = pd.DataFrame()
cor_values = df.T  #drop(['measure_year', 'measure_month', 'measure_day'], axis=1).T

for _, d1 in cor_values.iterrows():
    for _, d2 in cor_values.iterrows():
        if d1.name != d2.name:
            print(d1.name, d2.name)
            sub_df = pd.concat([d1, d2], axis=1).dropna()
            subcor = np.correlate(*(sub_df.values.T), 'full')
            
            shift_df.loc[d1.name, d2.name] = np.argmax(subcor) - subcor.shape[0]/2
shift_df

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
im = ax.imshow(shift_df.values)
im.set_clim(-1, 1)

ax.xaxis.set(ticks=list(range(len(shift_df.columns))), ticklabels=shift_df.columns)
plt.xticks(rotation=90)
ax.yaxis.set(ticks=list(range(len(shift_df.columns))), ticklabels=shift_df.columns)

cbar = ax.figure.colorbar(im, ax=ax, format='% .2f')
plt.show()

# Explain np.correlate

np.correlate tells how much `b` is forward of `a`

In [ ]:
x = np.linspace(0, 30, 300)
a = np.sin((x/3))
b = np.cos((x/3)+1.*np.pi)
# b = np.cos((x/3Z)+.1*np.pi)

plt.plot(x, a)
plt.plot(x, b)
plt.title('original comparison')
plt.show()

In [ ]:
cor = np.correlate(a, b, 'full')
xc = np.linspace(0, cor.size, cor.size)
xc -= xc.mean()
xc = xc.round().astype(int)
plt.plot(xc, cor)
plt.title('correlation')
plt.show()
shift = xc[np.argmax(cor)]
shift

In [ ]:
xb = x+shift*np.mean(np.diff(x))

plt.plot(x, a)
plt.plot(xb, b)
plt.title(f'shifted b {shift} samples forward to match')
plt.show()

# Split in features and target

In [ ]:
ddf = df.describe().T.sort_values('count')
n = 0.9
subset = ddf.loc[ddf['count'] > max(ddf['count'].values)*n]
subdf = df[subset.index[::-1]].dropna()
desdf = subdf.describe().T
desdf.index = pd.MultiIndex.from_tuples(map(lambda x: x.replace('_ch', '(ch)').replace('_de', '(de)').split('_'), desdf.index.values.tolist()))
desdf.sort_index()

In [ ]:
target = subdf.pop('arnhem_waterlevel')
features_name, target_name = subdf.columns.values, target.name
features, target = subdf.values, target.values
features.shape, target.shape

In [ ]:
index = pd.DatetimeIndex([datetime(y, m, d) for y, m, d in subdf[['measure_year', 'measure_month', 'measure_day']].values])

In [ ]:
plt.plot(index, target)
plt.title(target_name)
plt.show()

In [ ]:
for name, values in zip(features_name, features.T):
    plt.plot(index, values)
    plt.title(name)
    plt.show()

# Feature exploration

No features with zero variance

In [ ]:
from sklearn.feature_selection import VarianceThreshold

sel = VarianceThreshold()
sel.fit_transform(features).shape, features.shape

Seems useless in this case

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression, r_regression

new_features = SelectKBest(f_regression, k=10).fit_transform(features, target)
new_features.shape, features.shape

In [ ]:
from sklearn.feature_selection import SelectPercentile

new_features = SelectPercentile(f_regression).fit_transform(features, target)
new_features.shape, features.shape

# Importance model

In [ ]:
from sklearn.linear_model import Ridge, Lasso

model = Ridge().fit(features, target)
importance = np.abs(model.coef_)
importance = model.coef_

In [ ]:
sorting = np.argsort(importance)

plt.barh(width=importance[sorting], y=features_name[sorting])
plt.title("Feature importances via coefficients")
plt.show()

In [ ]:
from sklearn.feature_selection import SelectFromModel

threshold = np.sort(importance)[-5] + 0.01

sfm = SelectFromModel(model, threshold=threshold).fit(features, target)
print(f"Features selected by SelectFromModel: {features_name[sfm.get_support()]}")

# Sequential selection

In [ ]:
from sklearn.feature_selection import SequentialFeatureSelector

sfs_forward = SequentialFeatureSelector(
    model, n_features_to_select=5, direction="forward"
).fit(features, target)

sfs_backward = SequentialFeatureSelector(
    model, n_features_to_select=5, direction="backward"
).fit(features, target)

print(
    "Features selected by forward sequential selection: "
    f"{features_name[sfs_forward.get_support()]}"
)
print(
    "Features selected by backward sequential selection: "
    f"{features_name[sfs_backward.get_support()]}"
)


# Imputation

SimpleImputer is quick but the distribution changes to much

KNNImputer is slow but the results are ....

In [3]:
from sklearn.impute import SimpleImputer, KNNImputer

In [4]:
df.dropna(subset=['arnhem_waterlevel'], inplace=True)

In [5]:
target = df.pop('arnhem_waterlevel')

In [6]:
target_nans = ~target.isna()
features, target = df.loc[target_nans], target.loc[target_nans]

In [7]:
ddf = features.describe().T.sort_values('count')

print('original', features.shape)
for main_n in range(10):
    subset = ddf.loc[ddf['count'] > np.max(ddf['count'].values) * (main_n/10)]
    subdf = features[subset.index[::-1]]

    for sub_n in range(11):
        ind = (~subdf.isna()).mean(axis=1) >= sub_n/10
        path = f'subsets//{main_n}-{sub_n}.csv'
        subdf.loc[ind].dropna(axis=1, how='all').to_csv(path)

original (91984, 41)


In [8]:
def knn_impute(formpath, topath):
    print(formpath)
    subset = pd.read_csv(formpath, index_col=0)

    imp_mean = KNNImputer(
        missing_values=np.nan,
        n_neighbors=270,
        keep_empty_features=True
    )

    pd.DataFrame(
        imp_mean.fit_transform(subset.values),
        index=subset.index,
        columns=subset.columns
    ).to_csv(topath)

In [9]:
import glob

files = glob.glob('subsets\\*.csv')

In [10]:
# knn_impute('subsets\\8-8.csv', 'imputed_subsets\\8-8.csv')
for file in files[::-1]:
    knn_impute(file, file.replace('subsets', 'imputed_subsets'))

subsets\9-9.csv
subsets\9-8.csv
subsets\9-7.csv
subsets\9-6.csv
subsets\9-5.csv
subsets\9-4.csv
subsets\9-3.csv
subsets\9-2.csv
subsets\9-10.csv
subsets\9-1.csv
subsets\9-0.csv
subsets\8-9.csv
subsets\8-8.csv
subsets\8-7.csv
subsets\8-6.csv
subsets\8-5.csv
subsets\8-4.csv
subsets\8-3.csv
subsets\8-2.csv
subsets\8-10.csv
subsets\8-1.csv
subsets\8-0.csv
subsets\7-9.csv
subsets\7-8.csv
subsets\7-7.csv
subsets\7-6.csv
subsets\7-5.csv
subsets\7-4.csv
subsets\7-3.csv
subsets\7-2.csv
subsets\7-10.csv
subsets\7-1.csv
subsets\7-0.csv
subsets\6-9.csv
subsets\6-8.csv
subsets\6-7.csv
subsets\6-6.csv
subsets\6-5.csv
subsets\6-4.csv
subsets\6-3.csv
subsets\6-2.csv
subsets\6-10.csv
subsets\6-1.csv
subsets\6-0.csv
subsets\5-9.csv
subsets\5-8.csv
subsets\5-7.csv
subsets\5-6.csv
subsets\5-5.csv
subsets\5-4.csv


In [ ]:
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (20,3)

In [ ]:
for name in df.columns[3:]:
    df_count, df_bin = np.histogram(df[name].dropna().values, bins=200, density=True)
    plt.stairs(df_count, df_bin, label='original', linewidth=2)

    for file in files:
        dv = pd.read_csv(file.replace('subsets', 'imputed_subsets'), index_col=0)

        try:
            dv_count, dv_bin = np.histogram(dv[name].values, bins=200, density=True)

            in_match = pd.concat([
                pd.Series(df_count, index=df_bin[:-1]),
                pd.Series(dv_count, index=dv_bin[:-1])
            ], axis=1)
            in_match = in_match.groupby(in_match.index.astype(int)).mean().dropna()
            match = np.min(np.corrcoef(*in_match.T.values)) > .998
            # match = '3:' in dn
            
            if match:
                plt.stairs(dv_count, dv_bin, label=file, linestyle='--', linewidth=.5)
        except KeyError:
            pass
    
    plt.title(name)
    plt.legend(ncols=10, loc=1)
    plt.show()
    # break

In [ ]:

index = pd.DatetimeIndex([datetime(y, m, d) for y, m, d in df[['measure_year', 'measure_month', 'measure_day']].values])

for name in df.columns:
    plt.plot(index, df[name])
    plt.title(name)
    plt.show()